In [2]:
import time
import numpy as np
import pandas as pd
import featuretools as ft
import catboost
from tqdm import tqdm_notebook
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.optimize
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, f1_score
from sklearn.externals import joblib
from category_encoders import CatBoostEncoder

In [3]:
train_raw = pd.read_csv("train_pre_31.csv")
X_test = pd.read_csv("test_pre_31.csv")
train_raw = train_raw.drop_duplicates(subset=train_raw.columns[~train_raw.columns.isin([
        'acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'mcc',
       'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', "loctm_min", 
       'conam_mean_by_loctm_min', 
       'conam_mean_by_loctm_sec', 
       'bacno_cano_no_recent_10_conam_pos_trend',
       'bacno_cano_no_recent_7_loctm_diff', 
       'bacno_cano_loctm_uique_count', 
       'bacno_cano_14_rolling_times_prime_time_fraud_cnt',
       'bacno_cano_10_times_sleep_time_cnt',
       'bacno_cano_10_sleep_time_conam_lower_3000_cnt',
       'bacno_cano_10_prime_time_fraud_conam_lower_3000_cnt', 
       'bacno_cano_no_recent_10_ecfg_sum', 'fraud_ind', "locdt",
        'flbmk_value_counts', 'hcefg_value_counts', 'flg_3dsmk_value_counts'
         'ecfg_value_counts', 'ovrlt_value_counts', 'insfg_value_counts'
        "conam_mean_by_train_test_acqic", "loctm_value_counts",
        "loctm_hour_value_counts", 'loctm_sec'])], keep='last')
test_raw = pd.read_csv("test.csv")

X = train_raw.drop('fraud_ind', axis=1)
y = train_raw['fraud_ind']

print(train_raw.shape)
print(X_test.shape)
print(test_raw.shape)

(1461618, 115)
(421665, 114)
(421665, 22)


## 資料處理

In [4]:
all_features = pd.concat([X, X_test], axis=0)
all_features.tail()

del all_features["locdt"]
del all_features['flg_3dsmk_value_counts']
del all_features['flbmk_value_counts']
del all_features['hcefg_value_counts']
del all_features['ecfg_value_counts']
del all_features['ovrlt_value_counts']
del all_features['insfg_value_counts']
del all_features["conam_mean_by_train_test_acqic"]

del all_features["loctm_value_counts"]
del all_features["loctm_hour_value_counts"]
del all_features['loctm_sec']

X = all_features[:-len(X_test)]
X_test = all_features[-len(X_test):]
print(X.shape)
print(X_test.shape)

(1461618, 103)
(421665, 103)


In [5]:
cat_colnames = ["bacno", "cano", "mchno", "acqic", "mcc", "ecfg", "insfg", "contp", 'etymd',
                "stocn", "scity", "stscd", "ovrlt", "flbmk", "hcefg", "csmcu", "flg_3dsmk"] # + high_cacol

categorical_feature_idx = [i for i, col in enumerate(X.columns) if col in cat_colnames]

In [7]:
model_type = 'lgb'
for col in X.columns[categorical_feature_idx]:
    # for catboost
    if model_type == 'cat':
        X[col] = X[col].astype('str')
        X_test[col] = X_test[col].astype('str')
        print(col)
        print(X[col].dtype)
    else:
        X[col] = X[col].astype('category')
        X_test[col] = X_test[col].astype('category')
        print(col)
        print(X[col].dtype)

acqic
category
bacno
category
cano
category
contp
category
csmcu
category
ecfg
category
etymd
category
flbmk
category
flg_3dsmk
category
hcefg
category
insfg
category
mcc
category
mchno
category
ovrlt
category
scity
category
stocn
category
stscd
category


In [8]:
# shuffle一下
# perm = np.random.permutation(len(X))
# X = X.iloc[perm].reset_index(drop=True) 
# y = y.iloc[perm].reset_index(drop=True)

# Seed = 111
sss = StratifiedKFold(n_splits=4, random_state=111, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

Train: [ 361384  361385  361386 ... 1461615 1461616 1461617] Test: [      0       1       2 ... 1449552 1449553 1449554]
Train: [      0       1       2 ... 1461615 1461616 1461617] Test: [ 361384  361385  361386 ... 1453573 1453574 1453575]
Train: [      0       1       2 ... 1461615 1461616 1461617] Test: [ 722768  722769  722770 ... 1457594 1457595 1457596]
Train: [      0       1       2 ... 1457594 1457595 1457596] Test: [1084151 1084152 1084153 ... 1461615 1461616 1461617]
(1096214, 103)
(1096214,)
(365404, 103)
(365404,)


## Catboost

In [15]:
# train不起來可能是因為cat index搞錯或有Na/ Cat對training set size很敏感?? -> 不，是去掉village好很多
# Catboost
model_cat = catboost.CatBoostClassifier(iterations=20000,
                              learning_rate=0.05,
                              depth=10,
                              eval_metric='AUC',
                              scale_pos_weight=2,
                              # random_seed=11,
                              # l2_leaf_reg=4,
                              # one_hot_max_size=100,
                              # rsm=0.5,
                              metric_period=100,
                              # plot=True,
                              # subsample=0.5,
                              early_stopping_rounds=100)
model_cat.fit(X_train, y_train, eval_set=(X_val, y_val), 
              cat_features=categorical_feature_idx, 
              use_best_model=True)
model_cat.save_model("pre_31_final_cat_v1.dump")

0:	test: 0.9641066	best: 0.9641066 (0)	total: 5.59s	remaining: 1d 7h 2m 48s


KeyboardInterrupt: 